# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [25]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV


In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S  TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S  TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False  A/0/S  TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False  F/1/S  TRAPPIST-1e  16.0  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0          0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
1        109.0        9.0          25.0   549.0    44.0       Juanna Vines   
2         43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
3          0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
4        303.0       70.0         151.0   565.0     2.0  Willy Santantines   

   Transported  
0        False  
1         True  
2        False  
3        False  
4         True

In [7]:
spaceship.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [8]:
spaceship.shape

(8693, 14)

In [9]:
spaceship.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [10]:
spaceship_cleaned = spaceship.dropna()

In [11]:
spaceship_cleaned.isnull().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
dtype: int64

In [12]:
spaceship_cleaned.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [13]:
#your code here
# Transformar la columna "Cabin"
spaceship_cleaned['Cabin'] = spaceship_cleaned['Cabin'].apply(lambda x: x[0] if pd.notna(x) else 'Unknown')

# Mostrar el DataFrame transformado
print("\nDataFrame transformado:")
print(spaceship_cleaned)


DataFrame transformado:
     PassengerId HomePlanet CryoSleep Cabin    Destination   Age    VIP  \
0        0001_01     Europa     False     B    TRAPPIST-1e  39.0  False   
1        0002_01      Earth     False     F    TRAPPIST-1e  24.0  False   
2        0003_01     Europa     False     A    TRAPPIST-1e  58.0   True   
3        0003_02     Europa     False     A    TRAPPIST-1e  33.0  False   
4        0004_01      Earth     False     F    TRAPPIST-1e  16.0  False   
...          ...        ...       ...   ...            ...   ...    ...   
8688     9276_01     Europa     False     A    55 Cancri e  41.0   True   
8689     9278_01      Earth      True     G  PSO J318.5-22  18.0  False   
8690     9279_01      Earth     False     G    TRAPPIST-1e  26.0  False   
8691     9280_01     Europa     False     E    55 Cancri e  32.0  False   
8692     9280_02     Europa     False     E    TRAPPIST-1e  44.0  False   

      RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               N

C:\Users\evacv\AppData\Local\Temp\ipykernel_30752\2572250410.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spaceship_cleaned['Cabin'] = spaceship_cleaned['Cabin'].apply(lambda x: x[0] if pd.notna(x) else 'Unknown')


In [14]:
spaceship_cleaned = spaceship_cleaned.drop(columns=['PassengerId', 'Name'])

In [15]:
spaceship_cleaned = pd.get_dummies(spaceship_cleaned, columns=spaceship_cleaned.select_dtypes(include=['object']).columns)
spaceship_cleaned

Age  RoomService  FoodCourt  ShoppingMall     Spa  VRDeck  Transported  \
0     39.0          0.0        0.0           0.0     0.0     0.0        False   
1     24.0        109.0        9.0          25.0   549.0    44.0         True   
2     58.0         43.0     3576.0           0.0  6715.0    49.0        False   
3     33.0          0.0     1283.0         371.0  3329.0   193.0        False   
4     16.0        303.0       70.0         151.0   565.0     2.0         True   
...    ...          ...        ...           ...     ...     ...          ...   
8688  41.0          0.0     6819.0           0.0  1643.0    74.0        False   
8689  18.0          0.0        0.0           0.0     0.0     0.0        False   
8690  26.0          0.0        0.0        1872.0     1.0     0.0         True   
8691  32.0          0.0     1049.0           0.0   353.0  3235.0        False   
8692  44.0        126.0     4688.0           0.0     0.0    12.0         True   

      HomePlanet_Earth  HomePlanet_Europa  HomePlanet_Mars  ...  Cabin_D  \
0                False               True            False  ...    False   
1                 True              False            False  ...    False   
2                False               True            False  ...    False   
3                False               True            False  ...    False   
4                 True              False            False  ...    False   
...                ...                ...              ...  ...      ...   
8688             False               True            False  ...    False   
8689              True              False            False  ...    False   
8690              True              False            False  ...    False   
8691             False               True            False  ...    False   
8692             False               True            False  ...    False   

      Cabin_E  Cabin_F  Cabin_G  Cabin_T  Destination_55 Cancri e  \
0       False    False    False    False                    False   
1       False     True    False    False                    False   
2       False    False    False    False                    False   
3       False    False    False    False                    False   
4       False     True    False    False                    False   
...       ...      ...      ...      ...                      ...   
8688    False    False    False    False                     True   
8689    False    False     True    False                    False   
8690    False    False     True    False                    False   
8691     True    False    False    False                     True   
8692     True    False    False    False                    False   

      Destination_PSO J318.5-22  Destination_TRAPPIST-1e  VIP_False  VIP_True  
0                         False                     True       True     False  
1                         False                     True       True     False  
2                         False                     True      False      True  
3                         False                     True       True     False  
4                         False                     True       True     False  
...                         ...                      ...        ...       ...  
8688                      False                    False      False      True  
8689                       True                    False       True     False  
8690                      False                     True       True     False  
8691                      False                    False       True     False  
8692                      False                     True       True     False  

[6606 rows x 25 columns]

In [16]:
numerical_spaceship = spaceship_cleaned.select_dtypes(include=['float64','bool'])
numerical_spaceship

Age  RoomService  FoodCourt  ShoppingMall     Spa  VRDeck  Transported  \
0     39.0          0.0        0.0           0.0     0.0     0.0        False   
1     24.0        109.0        9.0          25.0   549.0    44.0         True   
2     58.0         43.0     3576.0           0.0  6715.0    49.0        False   
3     33.0          0.0     1283.0         371.0  3329.0   193.0        False   
4     16.0        303.0       70.0         151.0   565.0     2.0         True   
...    ...          ...        ...           ...     ...     ...          ...   
8688  41.0          0.0     6819.0           0.0  1643.0    74.0        False   
8689  18.0          0.0        0.0           0.0     0.0     0.0        False   
8690  26.0          0.0        0.0        1872.0     1.0     0.0         True   
8691  32.0          0.0     1049.0           0.0   353.0  3235.0        False   
8692  44.0        126.0     4688.0           0.0     0.0    12.0         True   

      HomePlanet_Earth  HomePlanet_Europa  HomePlanet_Mars  ...  Cabin_D  \
0                False               True            False  ...    False   
1                 True              False            False  ...    False   
2                False               True            False  ...    False   
3                False               True            False  ...    False   
4                 True              False            False  ...    False   
...                ...                ...              ...  ...      ...   
8688             False               True            False  ...    False   
8689              True              False            False  ...    False   
8690              True              False            False  ...    False   
8691             False               True            False  ...    False   
8692             False               True            False  ...    False   

      Cabin_E  Cabin_F  Cabin_G  Cabin_T  Destination_55 Cancri e  \
0       False    False    False    False                    False   
1       False     True    False    False                    False   
2       False    False    False    False                    False   
3       False    False    False    False                    False   
4       False     True    False    False                    False   
...       ...      ...      ...      ...                      ...   
8688    False    False    False    False                     True   
8689    False    False     True    False                    False   
8690    False    False     True    False                    False   
8691     True    False    False    False                     True   
8692     True    False    False    False                    False   

      Destination_PSO J318.5-22  Destination_TRAPPIST-1e  VIP_False  VIP_True  
0                         False                     True       True     False  
1                         False                     True       True     False  
2                         False                     True      False      True  
3                         False                     True       True     False  
4                         False                     True       True     False  
...                         ...                      ...        ...       ...  
8688                      False                    False      False      True  
8689                       True                    False       True     False  
8690                      False                     True       True     False  
8691                      False                    False       True     False  
8692                      False                     True       True     False  

[6606 rows x 25 columns]

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [17]:
#your code here
features = numerical_spaceship.drop(columns = ["Transported"])
target = numerical_spaceship["Transported"]

In [18]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

In [21]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

#

Initial Accuracy: 0.7866868381240545


- Evaluate your model

In [23]:
#your code here
# Evaluate the initial model
y_pred = rf.predict(X_test)
initial_accuracy = accuracy_score(y_test, y_pred)
print(f'Initial Accuracy: {initial_accuracy}')

Initial Accuracy: 0.7866868381240545


In [26]:
knn = KNeighborsClassifier(n_neighbors=3)

In [27]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [28]:
pred = knn.predict(X_test)
pred

array([ True,  True,  True, ...,  True,  True,  True])

In [29]:
accuracy_score(y_test, pred)

0.7549167927382754

**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [31]:
#your code here
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

- Run Grid Search

In [32]:
# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print(f'Best Parameters: {grid_search.best_params_}')
print(f'Best Cross-validation Score: {grid_search.best_score_}')


Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best Parameters: {'bootstrap': True, 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 200}
Best Cross-validation Score: 0.8041249175769043


- Evaluate your model

In [33]:
best_rf = grid_search.best_estimator_
y_pred_best = best_rf.predict(X_test)

# Evaluate the best model
best_accuracy = accuracy_score(y_test, y_pred_best)
print(f'Best Model Accuracy: {best_accuracy}')

Best Model Accuracy: 0.7866868381240545
